In [1]:
import pandas as pd
import numpy as np
from torchvision import datasets


In [2]:


# Path to the dataset folder, not a single class folder
dataset_path = r"C:\Users\vidia\OneDrive\Documents\mlp_proj\mlp_assignment\data\dataset-resized"
dataset_path2 = r"C:\Users\vidia\OneDrive\Documents\mlp_proj\mlp_assignment\data\realwaste-main\RealWaste"
# Load dataset correctly
trashNet_dataset = datasets.ImageFolder(root=dataset_path)
realWaste_dataset = datasets.ImageFolder(root=dataset_path2)


In [3]:
class_mapping = {
"cardboard": "cardboard",  
"Cardboard": "cardboard",  
"Glass": "glass",
"glass": "glass",
"Metal": "metal",
"metal": "metal",
"paper": "paper",
"Paper": "paper",
"plastic": "plastic",
"Plastic": "plastic",
"trash": "trash",
"Food Organics": "trash",
"Miscellaneous Trash": "trash",
}
 

In [27]:
# from torch.utils.data import Dataset
 
# class MergedDataset(Dataset):
#     def __init__(self, datasets, class_mapping):
#         self.samples = []
#         self.class_mapping = class_mapping
#         self.new_classes = sorted(set(class_mapping.values()))  # Unique merged class names
#         self.class_to_idx = {cls: i for i, cls in enumerate(self.new_classes)}  # New class indices
 
#         # Process each dataset
#         for dataset in datasets:
#             for img_path, label in dataset.samples:
#                 original_class = dataset.classes[label]
#                 if original_class in class_mapping:
#                     new_label = self.class_to_idx[class_mapping[original_class]]
#                     self.samples.append((img_path, new_label))
 
#         self.transform = datasets[0].transform  # Use the same transform
 
#     def __len__(self):
#         return len(self.samples)
 
#     def __getitem__(self, idx):
#         img_path, label = self.samples[idx]
#         image = datasets.folder.default_loader(img_path)  # Load image
#         if self.transform:
#             image = self.transform(image)
#         return image, label

In [ ]:
merged_dataset = MergedDataset([trashNet_dataset, realWaste_dataset], class_mapping)
 
# Create DataLoader
from torch.utils.data import DataLoader
train_loader = DataLoader(merged_dataset, batch_size=32, shuffle=True)
 
# Print new class names
print("Merged Classes:", merged_dataset.new_classes)

In [4]:
import numpy as np
import PIL.Image as Image
import matplotlib.pylab as plt
import tensorflow as tf
import tensorflow_hub as hub

In [36]:
batch_size = 32
img_height = 256
img_width = 256
data_root = 'dataset_augmented/'

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  str(data_root),
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)
 

Smallest width: 512, Smallest height: 384


In [35]:
import tensorflow as tf
import os
import shutil


data_root1 = r"C:\Users\vidia\OneDrive\Documents\mlp_proj\mlp_assignment\data\dataset-resized"
data_root2 = r"C:\Users\vidia\OneDrive\Documents\mlp_proj\mlp_assignment\data\realwaste-main\RealWaste"
# Extract class names from both datasets
classes1 = sorted(os.listdir(data_root1))
classes2 = sorted(os.listdir(data_root2))

# Print detected class names
print("Classes in dataset 1:", classes1)
print("Classes in dataset 2:", classes2)

# Manual mapping if class names differ
class_mapping = {
"cardboard": "cardboard",  
"Cardboard": "cardboard",  
"Glass": "glass",
"glass": "glass",
"Metal": "metal",
"metal": "metal",
"paper": "paper",
"Paper": "paper",
"plastic": "plastic",
"Plastic": "plastic",
"trash": "trash",
"Food Organics": "trash",
"Miscellaneous Trash": "trash",
}
 
# Create a unified directory structure
unified_dataset_path = 'dataset_merged/'
os.makedirs(unified_dataset_path, exist_ok=True)

# Function to copy & relabel images into a unified dataset
def copy_and_relabel_images(source_folder, target_folder, mapping):
    for original_class in os.listdir(source_folder):
        if original_class in mapping:  # Only process known classes
            new_class = mapping[original_class]
            source_class_path = os.path.join(source_folder, original_class)
            target_class_path = os.path.join(target_folder, new_class)

            os.makedirs(target_class_path, exist_ok=True)  # Create target class folder if not exist
            
            for img_name in os.listdir(source_class_path):
                source_img_path = os.path.join(source_class_path, img_name)
                target_img_path = os.path.join(target_class_path, img_name)
                shutil.copy2(source_img_path, target_img_path)  # Copy image to correct folder

# Copy images from both datasets into the unified dataset
copy_and_relabel_images(data_root1, unified_dataset_path, class_mapping)
copy_and_relabel_images(data_root2, unified_dataset_path, class_mapping)

print("Dataset merging and relabeling completed!")

# Now, use `image_dataset_from_directory` on the new merged dataset
batch_size = 32
img_height = 256
img_width = 256

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    unified_dataset_path,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size
)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    unified_dataset_path,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size
)


Classes in dataset 1: ['cardboard', 'glass', 'metal', 'paper', 'plastic', 'trash']
Classes in dataset 2: ['Cardboard', 'Food Organics', 'Glass', 'Metal', 'Miscellaneous Trash', 'Paper', 'Plastic', 'Textile Trash', 'Vegetation']
Dataset merging and relabeling completed!
Found 6525 files belonging to 6 classes.
Using 5220 files for training.
Found 6525 files belonging to 6 classes.
Using 1305 files for validation.


In [7]:
# Create DataLoader
from torch.utils.data import DataLoader
train_loader = DataLoader(train_ds, batch_size=32, shuffle=True)
val_dataloader = DataLoader(val_ds, batch_size=32, shuffle=True)

In [37]:
import os
import shutil
import random

# Paths
merged_dataset_path = "dataset_merged"
split_dataset_path = "dataset_split"
train_path = os.path.join(split_dataset_path, "train")
val_path = os.path.join(split_dataset_path, "val")

# Ensure directories exist
os.makedirs(train_path, exist_ok=True)
os.makedirs(val_path, exist_ok=True)

# Function to split dataset
def split_dataset(source_path, train_dest, val_dest, val_ratio=0.2):
    """Splits images into training and validation folders."""
    for class_name in os.listdir(source_path):
        class_source_path = os.path.join(source_path, class_name)
        class_train_path = os.path.join(train_dest, class_name)
        class_val_path = os.path.join(val_dest, class_name)

        # Create class directories if they don't exist
        os.makedirs(class_train_path, exist_ok=True)
        os.makedirs(class_val_path, exist_ok=True)

        # Get all images
        images = os.listdir(class_source_path)
        random.shuffle(images)  # Shuffle for randomness

        # Compute split sizes
        val_size = int(len(images) * val_ratio)
        train_images = images[val_size:]
        val_images = images[:val_size]

        # Move images to respective folders
        for img in train_images:
            shutil.move(os.path.join(class_source_path, img), os.path.join(class_train_path, img))
        
        for img in val_images:
            shutil.move(os.path.join(class_source_path, img), os.path.join(class_val_path, img))

        print(f"Class {class_name}: {len(train_images)} training, {len(val_images)} validation")

# Apply function
split_dataset(merged_dataset_path, train_path, val_path)

print("Train and validation split completed!")


Class cardboard: 692 training, 172 validation
Class glass: 737 training, 184 validation
Class metal: 960 training, 240 validation
Class paper: 876 training, 218 validation
Class plastic: 1123 training, 280 validation
Class trash: 835 training, 208 validation
Train and validation split completed!


In [8]:
unified_dataset_path = 'dataset_merged/'
merged_dataset = datasets.ImageFolder(root=unified_dataset_path)

In [9]:
from collections import Counter
 
# Count occurrences of each class
class_counts = Counter(label for _, label in merged_dataset)
 
# Print counts for each class
for class_name, class_idx in merged_dataset.class_to_idx.items():
    print(f"{class_name}: {class_counts[class_idx]}")

cardboard: 864
glass: 921
metal: 1200
paper: 1094
plastic: 1403
trash: 1043
